In [4]:
import numpy as np
import pandas as pd

In [5]:
def get_distance(x, y):
    # Евклидово расстояние
    return np.sqrt(np.sum(np.power(x-y, 2)))

In [47]:
class Classifier():
    def __init__(self):
        pass

    def fit(self, data, categorical_features_indexes):
        self.categorical_features_indexes = categorical_features_indexes
        self.categorical_features_values = {}
        for i in self.categorical_features_indexes:
            self.categorical_features_values.setdefault(i, list(set(data[:,i])))

        data = self.transform_data(data)
        self.data_x = np.array(data[:, :-1], dtype=float)
        self.data_y = data[:, -1]
        normalized_data = np.zeros(self.data_x.shape)
        self.min_max_values = {}
        for i, column in enumerate(self.data_x.transpose()):
            _min = None
            _max = None
            if i not in self.categorical_features_indexes:
                _min = np.min(column)
                _max = np.max(column)
            else:
                _min = 0
                _max = 1
            self.min_max_values.setdefault(i, [0, 1])

            for j, value in enumerate(column):
                normalized_data[j][i] = (value - _min) / (_max - _min)
        self.data_x = normalized_data

    def predict(self, x, k=3):
        x = self.normalize_element(self.transform_element(x))
        distances = [[i, get_distance(x, d)] for i, d in
                     enumerate(self.data_x)]
        neighbours = [ a[0] for a in sorted(distances, key=lambda x: x[1]) ][:k]
        cls = [self.data_y[i] for i in neighbours]
        return max([[c, cls.count(c)] for c in set(cls)], key=lambda x: x[1])[0]

    def normalize_element(self, element):
        res = np.zeros(element.shape)
        for i, value in enumerate(element):
            _min = self.min_max_values[i][0]
            _max = self.min_max_values[i][1]
            res[i] = (value - _min) / (_max - _min)
        return res

    def transform_categorical_value(self, index, value):
        # Преобразование значения категориального признака с помощью
        # one-hot encoding
        result = list(np.zeros(len(self.categorical_features_values[index])))
        try:
            result[self.categorical_features_values[index].index(value)] = 1
        except:
            # Если было подано значение категориального признака,
            # которого не было в обучающей выборке, то оставить все
            # значения нулями
            pass
        return result

    def transform_element(self, element):
        result = []
        for i, value in enumerate(element):
            if i in self.categorical_features_values:
                result.extend(self.transform_categorical_value(i, value))
            else:
                result.append(value)
        return np.array(result)

    def transform_data(self, data):
        result = []
        for element in data:
            result.append(list(self.transform_element(element)))
        return np.array(result)

In [48]:
raw_data = np.array(pd.read_csv('data.csv'))[:,1:]
raw_data

array([['МО', 'Весы', 'Красный', 0, 7.0, 8.0, 'Ч'],
       ['СВАО', 'Близнецы', 'Синий', 0, 8.0, 8.0, 'Ч'],
       ['СВАО', 'Весы', 'Салатовый', 0, 10.0, 8.0, 'Ч'],
       ['ВАО', 'Весы', 'Синий', 1, 8.0, 8.0, 'Ч'],
       ['ВАО', 'Стрелец', 'Черный', 1, 8.0, 6.5, 'К'],
       ['МО', 'Рыбы', 'Синий', 0, 8.0, 7.0, 'Ч'],
       ['СЗАО', 'Овен', 'Белый', 1, 12.0, 7.0, 'К'],
       ['САО', 'Близнецы', 'Зеленый', 0, 8.0, 8.0, 'Ч'],
       ['МО', 'Козерог', 'Зеленый', 0, 8.0, 7.0, 'Ч'],
       ['ЮЗАО', 'Телец', 'Голубой', 1, 7.0, 6.0, 'К'],
       ['ЗАО', 'Стрелец', 'Индиго', 0, 10.0, 7.0, 'Ч'],
       ['ЮЗАО', 'Близнецы', 'Черный', 1, 14.0, 6.0, 'К'],
       ['ЮВАО', 'Стрелец', 'Черный', 1, 6.0, 7.0, 'К'],
       ['ЮЗАО', 'Овен', 'Зеленый', 1, 8.5, 7.0, 'Ч'],
       ['ВАО', 'Телец', 'Черный', 1, 8.5, 7.0, '0'],
       ['ЮЗАО', 'Водолей', 'Оранжевый', 0, 11.0, 8.0, 'Ч'],
       ['МО', 'Дева', 'Черный', 0, 14.0, 10.0, 'Ч'],
       ['ЮВАО', 'Козерог', 'Зеленый', 0, 9.0, 8.0, 'К'],
       ['ЮВА

In [49]:
categorical_features_indexes = [0, 1, 2]

In [51]:
knn = Classifier()
knn.fit(raw_data, categorical_features_indexes)

In [52]:
[knn.predict(x[:-1], 5) for x in raw_data]

['Ч',
 'Ч',
 'Ч',
 'Ч',
 'К',
 'Ч',
 'Ч',
 'Ч',
 'К',
 'Ч',
 'Ч',
 'Ч',
 'К',
 'Ч',
 'К',
 'Ч',
 'Ч',
 'К',
 'К',
 'Ч',
 'Ч',
 'К']

In [53]:
knn.predict(['СВАО', 'Рыбы', 'Белый', 1, 9, 5])

'К'

In [2]:
list(zip([1,2], [3,4]))

[(1, 3), (2, 4)]